In [256]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (train_test_split, GroupKFold)
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.preprocessing import (OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler)
from sklearn.metrics import (classification_report,f1_score, recall_score, precision_score)


import warnings
warnings.filterwarnings('ignore')

## Основные сведенья после ДЗ1
Стало ясно, что целевая переменная (Credit_Score) обладает дисбалансом классов, поэтому будем на это обращать внимание. В качестве безлайна возьмем решающие деревья.

In [280]:
df = pd.read_csv('data/prep_data.csv')
df.head()

,Customer_ID,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Credit_History_Age_Months
0,CUS_0xd40,January,23,Scientist,19114.12,1824.843333,3,4,3,4,...,Unknown,810.0,26.822620,No,49.574949,80.415295,High_spent_Small_value_payments,312.5,Good,265
1,CUS_0xd40,February,23,Scientist,19114.12,1592.843333,3,4,3,4,...,Good,810.0,31.944960,No,49.574949,118.280222,Low_spent_Large_value_payments,284.8,Good,0
2,CUS_0xd40,April,23,Scientist,19114.12,1592.843333,3,4,3,4,...,Good,810.0,31.377862,No,49.574949,199.458074,Low_spent_Small_value_payments,223.5,Good,268
3,CUS_0xd40,May,23,Scientist,19114.12,1824.843333,3,4,3,4,...,Good,810.0,24.797347,No,49.574949,41.420153,High_spent_Medium_value_payments,341.5,Good,269
4,CUS_0xd40,June,23,Scientist,19114.12,1592.843333,3,4,3,4,...,Good,810.0,27.262259,No,49.574949,62.430172,NaN,340.5,Good,270


### Анализируем данные

In [259]:
# Наличие пустых значений в датасете 

missing_count = df.isnull().sum()
value_count = df.isnull().count()
missing_percentage = round(missing_count / value_count * 100, 2)
missing_df = pd.DataFrame({"count": missing_count, "percentage": missing_percentage})
missing_df

,count,percentage
Customer_ID,0,0.00
Month,0,0.00
Age,0,0.00
Occupation,0,0.00
Annual_Income,0,0.00
Monthly_Inhand_Salary,0,0.00
Num_Bank_Accounts,0,0.00
Num_Credit_Card,0,0.00
Interest_Rate,0,0.00
Num_of_Loan,0,0.00


In [260]:
# посчитаем в процентном соотношение сколько пропущеных данных есть 

skip_count = df.isna().sum().sum()
percent = (skip_count / df.shape[0]) * 100

print(f'Пропущенных данных составляет {round(percent,2)}%')

Пропущенных данных составляет 20.4%


Избавимся от пропущенных значений, так как данных после удаления вполне хватит для обучения

In [261]:
def data_info(df, word):
    print(df['Credit_Score'].value_counts())
    print(f'Размер df {word} удаления {df.shape[0]}')
    print()

data_info(df, 'до')
df = df.dropna()
data_info(df, 'после')

Credit_Score
Standard    51724
Poor        28205
Good        17295
Name: count, dtype: int64
Размер df до удаления 97224

Credit_Score
Standard    41970
Poor        22734
Good        13990
Name: count, dtype: int64
Размер df после удаления 78694



## Делим набор данных на train и test
Для сохранения пропорциональности распределения классов целевой переменной будем данные стратифицировать.

In [262]:
df_train, df_test = train_test_split(df, test_size=0.4, random_state=22, stratify=df['Credit_Score'])

Удалим не нужные признаки Customer_ID и Month

In [263]:
df_train.drop(columns=['Customer_ID','Month'], inplace=True)
df_test.drop(columns=['Customer_ID','Month'], inplace=True)

### Преобразуем категориальные признаки в числовые
1) Можно обратить внимание, что фичи <code>Credit_Mix</code>, <code>Payment_Behaviour</code> и целевая переменная<code>Credit_Score</code>  
обладают неким порядком, поэтому уместно использовать <code>Ordinal Encoding</code>.    
2) Для фичи <code>Occupation</code> лучше всего использовать <code>LeaveOneOut</code>, поскольку данная фича  
принимает 16 категорий и важно уменьшить утечку информации (кажется, что все encoder-ы типа Ohe не подойдут, так как они просто раздуют df, что для решающего дерева не совсем хороший вариант).
3) Для <code>Payment_of_Min_Amount</code> проблемой не будет использовать Ohe (всего классов два).

Выделим числовые и строковые типы данных.

In [264]:
digital_column = df_train.select_dtypes([np.number]).columns
string_column = df_train.select_dtypes([object]).columns

df_train[string_column].head()

,Occupation,Credit_Mix,Payment_of_Min_Amount,Payment_Behaviour,Credit_Score
87674,Journalist,Unknown,Yes,Low_spent_Small_value_payments,Standard
39369,Entrepreneur,Good,No,High_spent_Large_value_payments,Good
44495,Unknown,Standard,Yes,Low_spent_Small_value_payments,Standard
93773,Teacher,Standard,No,Low_spent_Medium_value_payments,Poor
82131,Architect,Unknown,No,High_spent_Large_value_payments,Standard


In [265]:
ordinal_features = ['Credit_Score', 'Credit_Mix','Payment_Behaviour']
ordinal_encoder = OrdinalEncoder(categories=[['Poor','Standard','Good'],
                                             ['Unknown','Bad','Standard', 'Good'],
                                             ['Unknown',
                                              'Low_spent_Small_value_payments',
                                              'Low_spent_Medium_value_payments',
                                              'Low_spent_Large_value_payments',
                                              'High_spent_Small_value_payments',
                                              'High_spent_Medium_value_payments',
                                              'High_spent_Large_value_payments']  
                                            ])

leave_one_out_encoder = ce.LeaveOneOutEncoder(cols=['Occupation'])

def category_encoder(df_new, topic: str):
    
    df = df_new.copy()
    df['Payment_of_Min_Amount'] = df['Payment_of_Min_Amount'].map({'Yes': 1, 'No': 0})

    if topic=='train':
        df[ordinal_features] = ordinal_encoder.fit_transform(df[ordinal_features])
        df['Occupation'] = leave_one_out_encoder.fit_transform(df['Occupation'], df['Credit_Score'])
    else:
        df[ordinal_features] = ordinal_encoder.transform(df[ordinal_features])
        df['Occupation'] = leave_one_out_encoder.transform(df['Occupation'])

    return df

df_train_new = category_encoder(df_train, 'train')
df_test_new = category_encoder(df_test, 'test')

Разбиваем данные на фичи и целевую переменную

In [266]:
def select_x_y(df):
    df_new = df.copy()
    x_new = df_new.drop(columns=['Credit_Score'], axis=1)
    y_new = df_new['Credit_Score']
    return x_new, y_new

x_train, y_train = select_x_y(df_train_new)
x_test, y_test = select_x_y(df_test_new)

## Реализация константного предсказания
Наиболее частотный класс для классификации

In [267]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=22)
dummy_clf.fit(x_train, y_train)

y_pred = dummy_clf.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      9094
         1.0       0.53      1.00      0.70     16788
         2.0       0.00      0.00      0.00      5596

    accuracy                           0.53     31478
   macro avg       0.18      0.33      0.23     31478
weighted avg       0.28      0.53      0.37     31478



## Безлайн (получим базовое качество)
Используем в качетсве безлайна решающие деревья.  
Основные премущество:  
- не чувствительны к масштабу признаков
- менее чуствительны к аномальным значениям

Попытаемся невилировать дисбаланс классов с помощью параметра <code>class_weight='balanced'</code>, который автоматически устанавливает веса, обратные частоте классов.   
Также рассмотрим подбор устойчивых метрик, которые учитывают дисбаланс классов. В качестве критерия разбивки возьмем "entropy", и добавим ограничения в глубину дерева и в количестве объектов в листе.   

In [268]:
model = DecisionTreeClassifier(
    criterion='entropy',
    class_weight='balanced', 
    min_samples_leaf=8, 
    max_depth=6,
    random_state=22)

model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.61      0.72      0.66      9094
         1.0       0.79      0.58      0.67     16788
         2.0       0.51      0.76      0.61      5596

    accuracy                           0.65     31478
   macro avg       0.63      0.69      0.65     31478
weighted avg       0.69      0.65      0.66     31478



- **Для poor(0)**: Основной акцент на recall, чтобы модель находила как можно больше клиентов с высоким риском.
- **Для standard(1)**: Поддержание баланса между recall и precision, так как клиенты из этой категории считаются промежуточными по риску. (не совсем ясно что подразумевается под стандартом)
- **Для good(2)**: Основной акцент на precision, чтобы минимизировать случаи, когда клиент ошибочно классифицируется как poor или standard.


### Интерпретация результатов

Если модель показывает высокий recall для категории poor, это значит, что она успешно выявляет большинство рискованных клиентов, что важно для управления рисками в кредитном скоринге. Если recall для poor низкий, это может означать о необходимости доработки модели, чтобы сократить пропуски рискованных заемщиков.

### Попробуем использовать ансамбль моделей для получения более высоких результатов
Создадим ансамбль моделей с помощью стэкинга. Так как ансамбль будет содержать линейные модели, поэтому стоит предообработать данные:
- стандартизировать данные c помощью гибрида <code>StandardScaler</code> и <code>RobustScaler</code>. Думаю, данный гибрид наиболее подходящий выбор для SVM и LogReg, так как эти алгоритмы:
    1) Чувствительны к масштабу данных.
    2) Наилучшим образом работают с нормально распределёнными признаками.
    3) Чувствительны к выбросам.
- используем стратегию взвешивания классов, чтобы модель уделяла больше внимания менее представленным классам

In [269]:
rf = RandomForestClassifier(random_state=22, class_weight='balanced')
svc = SVC(random_state=22, class_weight='balanced')
lr = LogisticRegression(random_state=22, class_weight='balanced')


preprocessor = Pipeline([
    ('robust_scaler', RobustScaler()),
    ('standard_scaler', StandardScaler())
])


final_estimator = Pipeline([
    ('scaler', preprocessor),
    ('lr', LogisticRegression(random_state=22, class_weight='balanced'))
])

model = StackingClassifier(
    estimators=[('rf', rf), ('svc', svc), ('lr', lr)],
    final_estimator=final_estimator,
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('stacking', model)
])

pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.75      0.75      0.75      9094
         1.0       0.80      0.75      0.77     16788
         2.0       0.63      0.76      0.69      5596

    accuracy                           0.75     31478
   macro avg       0.73      0.75      0.74     31478
weighted avg       0.76      0.75      0.75     31478



Можно увидеть, что ансамбль моделей показал лучший результат по сравнению с бейзлайном.

## Попробуем оценить бейзлайн через кросс-валидацию

Для корректной валидации необходимо:

- Сгруппировать данные по Customer_ID, чтобы данные одного клиента   
не попадали одновременно в train и test.
- Учитывать временную компоненту Month, разделяя данные так, чтобы более поздние месяцы   
всегда попадали в тестовую выборку.


### Преобразуем все категориальные признаки в числовые. 
Основные положения:
- Впринципе, основная логика сохраняется и дополняется преобразованием  
<code>Customer_ID</code> и <code>Month</code>. 
1) Month преобразуем через Oridinal Encoding,   
поскольку переменная содержит перечисление месяцев   
(на этапе обучение модели данная фича будет удалена,  
так как она нам нужна для сортировки). 
2) Customer_ID будет преобразована с помощью   
LableEncoding, поскольку данная фича будет использоваться для группировки, но не для обучения.

In [270]:
# копируем оригинальный df
df_new = df.copy()

month_order = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 
    'May': 5, 'June': 6, 'July': 7, 'August': 8, 
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}
label_encoder = LabelEncoder()


df_new['Month'] = df_new['Month'].map(month_order)
df_new['Customer_ID'] = label_encoder.fit_transform(df_new['Customer_ID'])

df_new = category_encoder(df_new, 'test')

Во время обучения мы будем удалять фичи Month и Customer_ID, поскольку они нужны будут для сортировки и группировки данных.    
При подсчете метрик все значения будут усреднятся с использованием подхода <code>average='weighted'</code>, чтобы учитывать дисбаланс классов.

In [271]:
# Сортировка данных по убыванию c начала 1-го месяца года
df_new = df_new.sort_values(by=['Month'])

X = df_new.drop(columns=['Credit_Score'], axis=1)
y = df_new['Credit_Score']
groups = df_new['Customer_ID']

# важно учитывать, чтобы данные одного клиента попали в одну группу
group_kfold = GroupKFold(n_splits=5)
splits = group_kfold.split(X, y, groups=groups)

model = DecisionTreeClassifier(
    criterion='entropy',
    class_weight='balanced',
    min_samples_leaf=8,
    max_depth=6,
    random_state=22
)

f1_scores = []
recall_scores = []
precision_scores = []

for train_idx, test_idx in splits:
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
 
    X_train.drop(columns=['Month', 'Customer_ID'], axis=1)
    X_test.drop(columns=['Month', 'Customer_ID'], axis=1)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    recall_scores.append(recall)
    precision_scores.append(precision)

print("F1-Score по каждому фолду:", f1_scores)
print("Среднее F1-Score:", np.mean(f1_scores))
print('*'*50)
print("Recall-Score по каждому фолду:", recall_scores)
print("Среднее Recall-Score:", np.mean(recall_scores))
print('*'*50)
print("Precision-Score по каждому фолду:", precision_scores)
print("Среднее Precision-Score:", np.mean(precision_scores))

F1-Score по каждому фолду: [np.float64(0.6521053027611898), np.float64(0.6592600649182315), np.float64(0.6549332272806824), np.float64(0.6636326981409807), np.float64(0.6647626057263797)]
Среднее F1-Score: 0.6589387797654929
**************************************************
Recall-Score по каждому фолду: [np.float64(0.6490882521125866), np.float64(0.6565855518139653), np.float64(0.6537899485354851), np.float64(0.6632568778194294), np.float64(0.6638073452789427)]
Среднее Recall-Score: 0.6573055951120818
**************************************************
Precision-Score по каждому фолду: [np.float64(0.7113066881873537), np.float64(0.7112626395797526), np.float64(0.7219873308963335), np.float64(0.7179248228462591), np.float64(0.7202102679103896)]
Среднее Precision-Score: 0.7165383498840177


В среднем усреднённые значения метрик будут совпадать с безлайном.

# Попробуем повысить качество модели

### Feature Engineering

#### <code>Income_Stability</code> = <code>Annual_Income</code>+<code>Monthly_Inhand_Salary </code>


Cоздадим новую фичу <code>Income_Stability</code>, которая показывает соотношение годового дохода к ожидаемому годовому доходу на основе ежемесячной зарплаты.

In [272]:
def create_Income_stability(df: pd.DataFrame):
    df['Income_Stability'] = (df['Monthly_Inhand_Salary'] * 12) / df['Annual_Income']
    df.drop(columns=['Annual_Income','Monthly_Inhand_Salary'], inplace=True)
    return df

df_train_new = create_Income_stability(df_train)
df_test_new = create_Income_stability(df_test)

### Expense_to_Balance_Ratio = <code>Total_EMI_per_month</code> + <code>Monthly_Balance</code> 

Эта новая фича будет интерпритировать финансовое состояние клиента,  
сколько расходов он имеет по сравнению с его ежемесячным балансом.

In [273]:
def create_Expense_to_Balance_Ratio(df: pd.DataFrame):
    df['Monthly_Balance'] = df['Monthly_Balance'].apply(lambda x: df['Monthly_Balance'].mean() if x == 0 else x)
    df['Expense_to_Balance_Ratio'] = df['Total_EMI_per_month'] / df['Monthly_Balance']

    df.drop(columns=['Total_EMI_per_month', 'Monthly_Balance'], inplace=True)
    return df

df_train_new = create_Expense_to_Balance_Ratio(df_train_new)
df_test_new  = create_Expense_to_Balance_Ratio(df_test_new)

### <code>Payment_Reliability_Score</code> = <code>Num_of_Delayed_Payment</code> + <code>Payment_of_Min_Amount</code>+<code> Payment_Behaviour</code>+<code>Delay_from_due_date</code>
Создадим новую фичу, которая будет интерпритировать общий показатель надежности клиента. Этот показатель будет учитывать количество просрочек, задолжностей по дням, платежное поведение и выполнение минимального платежа.

In [274]:
def calculate_payment_reliability(df):
    score = 100

    def payment_behaviour_score(behaviour):
        if "High_spent" in behaviour:
            if "Small_value" in behaviour:
                return -5
            elif "Medium_value" in behaviour:
                return -10 
            elif "Large_value" in behaviour:
                return -15 
        elif "Low_spent" in behaviour:
            if "Small_value" in behaviour:
                return 5  
            elif "Medium_value" in behaviour:
                return 0  
            elif "Large_value" in behaviour:
                return -5 
        elif "Unknown" in behaviour:
            return -10  
        else:
            return 0  

    score -= df['Delay_from_due_date'].apply(lambda x: abs(x) * 0.5 if x < 0 else x * 0.5)
    score -= df['Num_of_Delayed_Payment'] * 2
    score += df['Payment_Behaviour'].apply(payment_behaviour_score) 
    score -= df['Payment_of_Min_Amount'].apply(lambda x: 5 if x == 'No' else 0)

    score = score.clip(0, 100)
    return score


def prepocessing_score(df):
    df_new = df.copy()
    df_new['Payment_Reliability_Score'] = calculate_payment_reliability(df_new)
    df_new.drop(columns=['Delay_from_due_date', 'Num_of_Delayed_Payment', 'Payment_Behaviour', 'Payment_of_Min_Amount'], inplace=True)
    return df_new


df_train_new = prepocessing_score(df_train_new)
df_test_new = prepocessing_score(df_test_new)

Посмотрим количество числовых и строковых столбцов после FE.

In [275]:
digital_column = df_train_new.select_dtypes([np.number]).columns
string_column = df_train_new.select_dtypes([object]).columns

print(f'Количество столбцов, содержащее числовые данные %s' %len(digital_column))
print(f'Количество столбцов, содержащее текстовые данные %s' %len(string_column)) 

Количество столбцов, содержащее числовые данные 14
Количество столбцов, содержащее текстовые данные 3


### Преобразуем категориальные признаки в числовые
Такое же кодирование как раньше

In [276]:
# посмотрим на все уникальные значения

unique_credit_score = df[string_column]['Credit_Score'].unique()
unique_credit_mix = df[string_column]['Credit_Mix'].unique()
unique_occupation = len(df[string_column]['Occupation'].unique())

print(f'Уникальные значения для:\nCredit Score: {unique_credit_score}\nCredit_Mix: {unique_credit_mix}\nOccupation: {unique_occupation}')

Уникальные значения для:
Credit Score: ['Good' 'Standard' 'Poor']
Credit_Mix: ['Unknown' 'Good' 'Standard' 'Bad']
Occupation: 16


In [277]:
ordinal_features = ['Credit_Score', 'Credit_Mix']
ordinal_encoder = OrdinalEncoder(categories=[['Poor','Standard','Good'],
                                             ['Unknown','Bad','Standard', 'Good']])
leave_one_out_encoder = ce.LeaveOneOutEncoder(cols=['Occupation'])

def category_encoder(df, topic: str):
    if topic=='train':
        df[ordinal_features] = ordinal_encoder.fit_transform(df[ordinal_features])
        df['Occupation'] = leave_one_out_encoder.fit_transform(df['Occupation'], df['Credit_Score'])
    else:
        df[ordinal_features] = ordinal_encoder.transform(df[ordinal_features])
        df['Occupation'] = leave_one_out_encoder.transform(df['Occupation'])

    return df

df_train_new = category_encoder(df_train_new, 'train')
df_test_new = category_encoder(df_test_new, 'test')

## Модель
Посмотрим на качество модели после всех преобразований

In [278]:
# Отделим целевую переменную от фичей

x_train, y_train = select_x_y(df_train_new)
x_test, y_test = select_x_y(df_test_new)

### Для проверки возьмем тот же безлайн

In [279]:
model = DecisionTreeClassifier(
    criterion='entropy',
    class_weight='balanced', 
    min_samples_leaf=8, 
    max_depth=6,
    random_state=22)

model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.61      0.67      0.64      9094
         1.0       0.77      0.55      0.64     16788
         2.0       0.44      0.76      0.56      5596

    accuracy                           0.62     31478
   macro avg       0.61      0.66      0.61     31478
weighted avg       0.67      0.62      0.63     31478



<code>Вывод</code>: FE только ухудшило качество модели. Безлайн с деревьями решений имеет хороший score, но не такой хороший как ансамбль моделей.